In [1]:
## EMU-SV: StateVector class testing api

import torch 
from IPython.display import Latex 

from emu_sv import StateVector, inner
from emu_sv.hamiltonian import RydbergHamiltonian
from emu_sv.dense_operator import DenseOperator
dtype = torch.complex128

import math

#### The basics function related to StateVector class in EMU-SV

We are going to define 2 states: 


$ |\psi\rangle = \frac{1}{\sqrt 2}(|00\rangle+|11\rangle)$ and $|\phi\rangle = \frac{1}{\sqrt 2}(|01\rangle+|11\rangle)$

In [2]:

factor = math.sqrt(2.0)

basis = ("r","g")
nqubits = 2
string_state1 = {"gg":1.0,"rr":1.0}
state1 = StateVector.from_state_string(basis=basis, nqubits=nqubits,strings=string_state1)


string_state2 = {"gr":1.0/factor,"rr":1.0/factor}
state2 = StateVector.from_state_string(basis=basis,nqubits=nqubits,strings=string_state2)

In [3]:
# representation of state 1
state1

tensor([0.7071+0.j, 0.0000+0.j, 0.0000+0.j, 0.7071+0.j],
       dtype=torch.complex128)

In [4]:
#shape of the tensor and sample
print("\nShape: ",state1.vector.shape[0])
#norm of the state
print("\nnorm:",state1.norm())
display(Latex(r"Sampling $|\psi\rangle$:"))
state1.sample() # sampling the state


Shape:  4

norm: tensor(1.0000, dtype=torch.float64)


<IPython.core.display.Latex object>

Counter({'00': 503, '11': 497})

In [5]:
display(Latex(r"Inner product $\langle\psi|\phi \rangle :$"))
print(inner(state1,state2).item())
display(Latex(r"norm of $|\psi\rangle:$ "))
print(state1.norm().item())
display(Latex(r"$ |\delta\rangle=|\phi\rangle+2 e^{\pi i} |\psi\rangle:$"))
result = state1 + 2*torch.exp(torch.tensor(3.14159j))*state2
print("\nFinal state:",result)
print("\nsampling the resulting state")
print(result.sample(1000))


<IPython.core.display.Latex object>

(0.4999999999999999+0j)


<IPython.core.display.Latex object>

0.9999999999999999


<IPython.core.display.Latex object>


Final state: tensor([ 0.7071+0.0000e+00j, -1.4142+3.5853e-06j,  0.0000+0.0000e+00j,
        -0.7071+3.5853e-06j], dtype=torch.complex128)

sampling the resulting state
Counter({'01': 660, '00': 174, '11': 166})


In [6]:
# sampling the other state
sampling = state2.sample(1000)
print(sampling)
sampling["11"]

Counter({'11': 516, '01': 484})


516

In [7]:
# sampling the state |111>+|000>
nqubits = 3
string_state = {"rrr":1.0, "ggg":1.0}
state3 = StateVector.from_state_string(basis=basis,nqubits=nqubits,strings=string_state)
state3.sample(1000)

Counter({'111': 513, '000': 487})

In [8]:
# sampling the state |111>+|000>
nqubits = 3
string_state = {"rrr":1.0, "ggg":1.0}
state3 = StateVector.from_state_string(basis=basis,nqubits=nqubits,strings=string_state)
state3.sample(1000)

Counter({'111': 515, '000': 485})

In [9]:
# 10 atoms in the ground state using the make function
n=10
ground = StateVector.make(n)
ground.sample()


Counter({'0000000000': 1000})

### Custom sparse Rydberg Hamiltonian

Implementation of Rydberg Hamiltonian in sparse format and multiplication with a random vector




In [10]:
N = 8
device = "cpu"

omega = torch.randn(N, dtype=dtype, device=device)
delta = torch.randn(N, dtype=dtype, device=device)
interaction_matrix = torch.randn((N, N))
v = torch.randn((2,)*N, dtype=dtype, device=device)
h_custom = RydbergHamiltonian(
    omegas=omega, deltas=delta, interaction_matrix=interaction_matrix, device=device
)
res_sparse = h_custom @ v
print("\nCustom multiplication: sparse matrix with random vector", res_sparse[0:10])


Custom multiplication: sparse matrix with random vector tensor([-1.9323-1.3664j,  0.2566+1.8306j, -0.3390+1.0845j, -0.2470-0.7743j,
         0.4235-0.8698j,  0.4089+0.2061j, -0.3048-0.3941j,  0.8510-4.2901j,
         0.9998-1.7926j,  3.1959+1.0241j], dtype=torch.complex128)


In [11]:
## use make in order to generate a ground state |000000..0>

StateVector.make(2)

tensor([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j], dtype=torch.complex128)

In [13]:
vector = torch.tensor([0.1,1.0,1.0,1.0])
state = StateVector(vector)
state._normalize()


tensor([0.0576+0.j, 0.5764+0.j, 0.5764+0.j, 0.5764+0.j],
       dtype=torch.complex128)

Operators in Emu-SV

Creating and algebra related to operators

In [ ]:
x = {"g"+"r":1.0, "r"+"g":1.0}
z = {"g"+"g":1.0, "r"+"r":-1.0}
operators = {"X": x, "Z": z}
operations = [
        (
            1.0,
            [
                ({"X": 2.0}, [0, 2]),
                ({"Z": 3.0}, [1]),
            ],
        )
    ]

basis = {"r","g"}
N= 3
oper_a = DenseOperator.from_operator_string(basis,N,operations,operators)
oper_a



In [36]:
expected_op = torch.zeros(2**N,2**N,dtype=torch.complex128)
expected_op[0,5] = 12.+0.j
expected_op[1,4] = 12.+0.j
expected_op[2,7]= -12.0+0j
expected_op[3,6] = -12.+0.j
expected_op[4,1] = 12.+0.j
expected_op[5,0] = 12.+0j
expected_op[6,3] = -12.+0.j
expected_op[7,2] = -12.+0.j

assert torch.allclose(expected_op,oper_a.matrix)

In [ ]:
x = {"g"+"r":1.0, "r"+"g":1.0}
y = {"g"+"r":-1.0j, "r"+"g":1.0j}
operators = {"X": x, "Y": y}
operations = [
        (
            2.0,
            [
                ({"X": 2.0}, [0, 2]),
                ({"Y": 3.0}, [1]),
            ],
        )
    ]

basis = {"r","g"}
N = 3
oper_b = DenseOperator.from_operator_string(basis,N,operations,operators)
oper_b 


In [ ]:
#summing 2 operators
oper_a+oper_b

In [ ]:
#scalar multiplication
5.0*oper_a

In [ ]:
#operator applied to a StateVector
state = StateVector.make(3) #|000>
oper_a * state

In [ ]:
# expectation value
expectation_000 = oper_a.expect(state)
expectation_000

In [ ]:
# aplication of an operator to a StateVector


basis = ("r", "g")
state = {"rrr": 1.0, "ggg": 1.0}
nqubits = 3
from_string = StateVector.from_state_string(
    basis=basis, nqubits=nqubits, strings=state
)

oper_a * from_string 

In [ ]:
#expectation value
print(oper_a.expect(from_string))
print(oper_b.expect(from_string))

In [ ]:
## operator - operator multipliation

oper_a @ oper_b

In [8]:
import random

lst= []

lst1 = random.sample(range(0,10),10)
lst.append(lst1)
lst2 = random.sample(range(0,10),10)
lst.append(lst2)

In [9]:
lst

[[0, 4, 7, 2, 3, 8, 6, 9, 1, 5], [2, 6, 1, 5, 3, 9, 8, 4, 7, 0]]